In [1]:
import sys
import data, data.gene_datasets, data.colombos
import numpy as np
import matplotlib, matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import data.graph
from data.graph import Graph
import analysis
from analysis import monitoring
import main
import optimization as otim
from torch.autograd import Variable

In [2]:
dataset = data.gene_datasets.TCGAGeneInference()
class Object(object):
    pass

opt = Object()
opt.seed = 1993
opt.nb_class = None
opt.nb_examples = None
opt.nb_nodes = None
opt.graph = "pathway"
opt.dataset = dataset
opt.add_self = True
opt.add_connectivity = False
opt.batch_size = 128

graph = Graph()
path = "/data/lisa/data/genomics/graph/pancan-tissue-graph.hdf5"
graph.load_graph(path)
graph.intersection_with(dataset)
g = nx.from_numpy_matrix(graph.adj)
mapping = dict(zip(range(0, len(dataset.df.columns)), dataset.df.columns))
g = nx.relabel_nodes(g, mapping)

Converting one-hot labels to integers


In [3]:
dataset.df.shape

(10459, 5000)

In [4]:
# Get the model
load_folder = "./experiments/experiments/tcga-tissue-gene-inference/cgn/testing123/329006fa5a858bcbac9ffe480bfc1485/1993"
opt.load_folder = load_folder
opt.load_checkpoint = True
opt.epoch = 100
opt.training_mode = 'gene-inference'
my_model, optimizer, epoch, saved_opt = analysis.monitoring.load_checkpoint(load_folder, 
                                                                           opt, dataset, graph)

=> loading checkpoint './experiments/experiments/tcga-tissue-gene-inference/cgn/testing123/329006fa5a858bcbac9ffe480bfc1485/1993/checkpoint.pth.tar'
Loading the model with these parameters: <__main__.Object object at 0x7f2fe6a82550>
=> loaded checkpoint './experiments/experiments/tcga-tissue-gene-inference/cgn/testing123/329006fa5a858bcbac9ffe480bfc1485/1993/checkpoint.pth.tar' (epoch 10)
Doing drop-out
Our model:
CGN (
  (emb): EmbeddingLayer (
  )
  (my_convs): ModuleList (
    (0): CGNLayer (
      (linear): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
      (eye_linear): Conv1d(16, 16, kernel_size=(1,), stride=(1,))
    )
  )
  (my_dropouts): ModuleList (
    (0): Dropout (p = 0.1)
  )
  (last_inference_layer): Conv1d(32, 1, kernel_size=(1,), stride=(1,))
)


In [34]:
# Get the dataset
train_set, valid_set, test_set = main.split_dataset(dataset, batch_size=1, seed = saved_opt.seed,
                                                   nb_samples=saved_opt.nb_examples, train_ratio=saved_opt.train_ratio, 
                                                    nb_per_class=saved_opt.nb_per_class)

In [50]:
reload(otim)

def evaluate(my_model, eval_set, opt):
    
    dataset = eval_set.dataset
    criterions = otim.get_criterion(dataset, opt.training_mode)
    
    results = []
    
    for no_b, mini in enumerate(eval_set):
            inputs, targets = mini['sample'], mini['labels']

            inputs = Variable(inputs, requires_grad=False).float()
            #targets = Variable(targets, requires_grad=False).long()

            if opt.cuda:
                inputs = inputs.cuda()
                targets = targets.cuda()

            # Forward pass: Compute predicted y by passing x to the model
            my_model.eval()

            y_pred = my_model(inputs)

            # Compute and print loss
            crit_loss = otim.compute_loss(criterions, y_pred, targets, opt.training_mode, opt.semi_mse_lambda)
            results.append(crit_loss.data.numpy()[0])
            
    return results
        

results = evaluate(my_model, valid_set, saved_opt)

In [51]:
results

[0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,
 0.15951988,